# Circe: Advanced - Creating Custom Encoders and Decoders (Extended)

## Setup

In [ ]:
import $ivy.`io.circe::circe-core:0.14.1`, $ivy.`io.circe::circe-generic:0.14.1`, $ivy.`io.circe::circe-parser:0.14.1`

## Creating Custom Encoders and Decoders

### Manual Creation

This style of codec creation is very verbose and a little bit annoying. It is, however, very low level and allows a great deal of flexibility. Usually, there is a better way.

In [ ]:
import io.circe.{Json => CirceJson, _}

case class Student(name: String, grade: Int)

implicit val encodeStudent: Encoder[Student] = new Encoder[Student] {
  final def apply(s: Student): CirceJson = CirceJson.obj(
    ("name", CirceJson.fromString(s.name)),
    ("grade", CirceJson.fromInt(s.grade))
  )
}

implicit val decodeStudent: Decoder[Student] = new Decoder[Student] {
  final def apply(c: HCursor): Decoder.Result[Student] = for {
    name  <- c.downField("name").as[String]
    grade <- c.downField("grade").as[Int]
  } yield {
    Student(name, grade)
  }
}

// Testing encoding and decoding
val student = Student("John", 90)
val encoded = encodeStudent(student)
val decoded = decodeStudent.decodeJson(encoded)

encoded
decoded

### Using Contramap

In [ ]:
import io.circe.syntax._
import io.circe.Encoder

case class Car(make: String, model: String)

implicit val encodeCar: Encoder[Car] = Encoder[String].contramap[Car](car => s"${car.make}-${car.model}")

val myCar = Car("Toyota", "Corolla")
val encodedCar = myCar.asJson

encodedCar

Here, we use `contramap` to convert a `Car` instance to a `String` before applying the original `String` encoder. This offers a more functional way of reusing existing encoders. And with this, the beauty of functional programming really comes into view: the existing `String` encoder allows us to skip the boilerplate and piggyback our encoder instance on top of an already defined encoder! This works with *any* encoder that is already defined, not just those provided by Circe.

### Using Map

In [ ]:
import io.circe.Decoder

implicit val decodeCar: Decoder[Car] = Decoder[String].map { str =>
  val parts = str.split('-')
  Car(parts(0), parts(1))
}

val decodedCar = decodeCar.decodeJson(encodedCar)
decodedCar

In this case, we use `map` to construct a `Car` from a `String`. We leverage the existing `String` decoder and then apply a function to convert it to our desired type. This offers a more functional and composable approach to defining decoders.

These techniques offer more composable and functional ways to define encoders and decoders. They allow for better reuse of existing instances and offer more concise definitions that can be far less taxing on the developer's most limited resource (according to Dijkstra): short term memory